In [149]:
import numpy as np
from scipy.spatial.transform import Rotation as Rot
from pathlib import Path
R = 50.0
Y = 300.0
X = 50
P = Y / (2*np.pi * R**2)
L = R * np.arccos(1 - P)

In [150]:
gold = (1 + np.sqrt(5)) / 2
idx = np.arange(0, X, dtype=float) + 0.5
theta = np.arccos(1 - 2 * idx / X)
phi = np.pi * (1 + np.sqrt(5)) * idx
centers = np.column_stack((np.full(X, R), theta, phi))
print(centers[:, 0] * np.sin(centers[:, 1]) * np.sin(centers[:, 2]))

[ -9.27360553   7.54798952   6.09164325 -21.80654114  28.07168512
 -18.52027398  -3.63178331  26.82210963 -37.55525983  28.3733029
  -2.72174983 -26.28971752  42.78608978 -36.95893933  10.87186618
  22.16019041 -44.49195343  43.70300626 -19.55550659 -15.55610954
  43.0544593  -48.12286944  27.76526094   7.43534208 -38.88232886
  49.89396638 -34.64260333   1.27061461  32.49861954 -48.86920653
  39.45319452  -9.62108397 -24.56408671  45.0825695  -41.57493507
  16.64931428  15.88592894 -38.73790762  40.4629055  -21.31614199
  -7.44213567  30.16731751 -35.5167097   22.32268311   0.49392287
 -19.66200667  25.44905238 -17.08464218   2.70578138   5.47223751]


In [151]:
res = 1.5 * Y
r_c = L/R
patches = []
for (R, p_i, a_i) in centers:

    inc = r_c/ (np.sqrt(res) - 1)
    for p in range(int(np.sqrt(res))):
        po = p * inc - r_c/2
        for j in range(int(np.sqrt(res))):
            inc_2 = 2*np.pi/ (int(np.sqrt(res)))
            az = j * inc_2
            pos: np.ndarray  = np.array([[R * np.cos(az)  * np.sin(po)], [R * np.sin(az) * np.sin(po)], [R * np.cos(po)]])
            r = np.array(Rot.from_rotvec([0, p_i , 0]).as_matrix())
            pos_n: np.array = r @ pos
            r = np.array(Rot.from_rotvec([0, 0, a_i]).as_matrix())
            pos_n = r @ pos_n
            if p < 5:
                print("---------")
                print(r)
                print(p_i * 180/np.pi)
                print(pos[1])
                print(pos_n[1][0])
            patches.append([pos_n[0][0], pos_n[1][0], pos_n[2][0]])
patches = np.array(patches)



---------
[[ 0.36237489  0.93203242  0.        ]
 [-0.93203242  0.36237489  0.        ]
 [ 0.          0.          1.        ]]
11.478340954533579
[-0.]
-4.766365949924068
---------
[[ 0.36237489  0.93203242  0.        ]
 [-0.93203242  0.36237489  0.        ]
 [ 0.          0.          1.        ]]
11.478340954533579
[-1.44018119]
-5.486523581790295
---------
[[ 0.36237489  0.93203242  0.        ]
 [-0.93203242  0.36237489  0.        ]
 [ 0.          0.          1.        ]]
11.478340954533579
[-2.75239595]
-6.539236313394288
---------
[[ 0.36237489  0.93203242  0.        ]
 [-0.93203242  0.36237489  0.        ]
 [ 0.          0.          1.        ]]
11.478340954533579
[-3.82004826]
-7.8309659987793
---------
[[ 0.36237489  0.93203242  0.        ]
 [-0.93203242  0.36237489  0.        ]
 [ 0.          0.          1.        ]]
11.478340954533579
[-4.54827252]
-9.246936786736516
---------
[[ 0.36237489  0.93203242  0.        ]
 [-0.93203242  0.36237489  0.        ]
 [ 0.          0.     

In [152]:
PPV = 2
N = int(PPV * (4/3) * np.pi * R**3)
sphere = np.random.uniform(-R, R, (N, 3))
sphere = sphere[np.linalg.norm(sphere, axis=1)<= R]

In [153]:
def save_dump(points, filename="out/patches.dump", box_len=1000):
    """
    Save coordinates to a dump file, for use with OVITO
    """
    num = sum(pt.shape[0] for pt in points)
    Path(filename).parent.mkdir(parents=True, exist_ok=True)
    with open(filename, "w") as f:
        f.write("ITEM: TIMESTEP\n0\n")
        f.write(f"ITEM: NUMBER OF ATOMS\n{num}\n")
        f.write(
            f"ITEM: BOX BOUNDS pp pp pp\n{-box_len // 2} {box_len // 2}\n{-box_len // 2} {box_len // 2}\n{-box_len//2} {box_len//2}\n"
        )
        f.write("ITEM: ATOMS id type x y z\n")
        for i in range(0, len(points)):
            for j, (x, y, z) in enumerate(points[i], start=1):
                f.write(f"{j} {i + 1} {x:.6f} {y:.6f} {z:.6f}\n")
        print("dumped to", filename)
save_dump([sphere, patches], box_len=2*R)


dumped to patches.dump
